In [13]:
# 04_agent_with_tools_notebook.py

import os
import uuid
import json
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

from IPython.display import display, Markdown, HTML
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent


# ---------------- Tools ----------------
@tool
def word_count(text: str) -> int:
    """Return the number of words in the given text."""
    return len([w for w in text.strip().split() if w])


# ---------------- Agent Setup ----------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

search = DuckDuckGoSearchRun()
tools = [search, word_count]

agent = create_agent(
    model=llm,
    tools=tools,
    debug=True,
    system_prompt=(
        "You are a tool-using assistant.\n"
        "- Use tools when helpful.\n"
        "- Summarize clearly.\n"
        "- Be accurate."
    ),
)

session_id = "notebook-session"


# ---------------- Notebook Runner ----------------
def run_agent(user_input: str):
    request_id = str(uuid.uuid4())[:8]
    timestamp = datetime.utcnow().isoformat() + "Z"

    display(Markdown(f"## 🧠 Agent Request"))
    display(Markdown(f"**Input:** `{user_input}`"))
    display(Markdown(f"**Session ID:** `{session_id}`  \n**Request ID:** `{request_id}`"))
    display(HTML("<hr>"))

    try:
        result = agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={
                "tags": ["notebook", "tools-agent"],
                "metadata": {"session_id": session_id, "request_id": request_id},
            },
        )

        messages = result.get("messages", [])
        answer = messages[-1].content if messages else str(result)

        # --- Output section ---
        display(Markdown("### Final Answer"))
        display(Markdown(answer))

        # --- Trace section ---
        trace_data = {
            "timestamp": timestamp,
            "session_id": session_id,
            "request_id": request_id,
            "model": "gpt-4o-mini",
            "tools_available": [t.name if hasattr(t, "name") else str(t) for t in tools],
        }

        display(Markdown("### 🔍 Trace Metadata"))
        display(Markdown(f"```json\n{json.dumps(trace_data, indent=2)}\n```"))

        display(HTML("<hr style='border:2px solid #ddd'>"))

    except Exception as e:
        display(Markdown("### Error"))
        display(Markdown(f"`{str(e)}`"))